# SLKB Pipeline

Here, we will go over the discussed pipeline using a Toy Data. Feel free to use this file to analyze your dataset. The file is divided into 3 main parts: (1) Data creation, (2) Score calculation, (3) Query Results

## Before getting started

Make sure an R environment with GEMINI, and mageck tool are located in your path. To see whether you can run their respective scores or not, you can run the following command:

```
import shutil
shutil.which('R') ## should yield accessed R environment location
shutil.which('mageck') ## should yield MAGeCK location
```

In additon, make sure to install SLKB python package. The details can be located at its [website](https://www.google.com)



In [1]:
## First, we load in our packages
import SLKB

import sqlite3
import numpy as np
import pandas as pd
import os
import pickle
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import subprocess
import shlex
import urllib

# setting warning to None
pd.set_option('mode.chained_assignment', None)

# Section 1 - Data Preperation

First, we start by installing a pickle file that contains the demo data (Pickle version 4). Not all input files are required. For score calculation, only sequences and counts files are sufficient. 

In [2]:
# taken from
#toy_data = urllib.request.urlopen(pickle_loc).read()
with open('demo_data.pickle', 'rb') as handle:
    toy_data = pickle.load(handle)

demo_data = SLKB.load_demo_data()
# sequences_ref = 
# counts_ref = urllib.request.urlopen(pickle_loc).read()
# scores_ref = urllib.request.urlopen(pickle_loc).read()

sequence_ref = demo_data['sequence_ref']
counts_ref = demo_data['counts_ref']
score_ref = demo_data['score_ref']

# # let us create a local sqlite3 database to store our results in, and connect to it
#db_engine = SLKB.create_SLKB(location = os.getcwd(), name = 'myCDKO_db')
#SLKB_engine = sqlalchemy.create_engine(db_engine)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gokbag.1/Documents/CodingProjectsLocal/SLKB-Analysis-Pipeline/SLKB_env/lib/python3.10/site-packages/SLKB/files/demo_data.pickle'

In [3]:
db_engine

'sqlite:///myCDKO_db'

In [4]:
print(counts_ref.columns)

Index(['guide_1', 'guide_2', 'gene_1', 'gene_2', 'count_replicates',
       'cell_line_origin', 'study_conditions', 'study_origin'],
      dtype='object')


In [5]:
print(sequence_ref.columns)

Index(['sgRNA_guide_name', 'sgRNA_guide_seq', 'sgRNA_target_name'], dtype='object')


In [6]:
print(score_ref.columns)

Index(['gene_1', 'gene_2', 'study_origin', 'cell_line_origin', 'SL_score',
       'SL_score_cutoff', 'statistical_score', 'statistical_score_cutoff'],
      dtype='object')


## Inserting to DB

After each data is prepared, the study is ready to be inserted into the database. The ```prepare_study_for_export``` function will go over the data and prepare the data for insertion. It will produce errors where necessary, make sure that your files match with the template. 

Make sure your control gene list is set up properly to correctly categorize the counts file. The counts file will produce a ```target_type``` column that contains three categories:
1. Dual - Both sgRNAs targeting different genes.
2. Single - Both sgRNAs targeting the same gene (i.e., gene_1 + gene_1, or gene_1 + control)
3. Control - Both sgRNAs targeting controls.

In [8]:
study_controls = ['0SAFE',
                 '0SAFE-SAFE-GE',
                 '0SAFE-SAFE-SP',
                 '0SAFE-SAFE-MP',
                 '0SAFE-SAFE-U2',
                 '0SAFE-SAFE-DTKP',
                 '0SAFE-SAFE-ACOC',
                 '0SAFE-SAFE-TMM',
                 '0SAFE-SAFE-U1',
                 '0SAFE-SAFE-U3']
study_conditions = [["T0_1", 
                     "T0_2"],
                    ["T12_1",
                     "T12_2"]]

db_inserts = SLKB.prepare_study_for_export(sequence_ref = sequence_ref.copy(), 
                                      counts_ref = counts_ref.copy(),
                                      score_ref = score_ref.copy(),
                                      study_controls = study_controls,
                                      study_conditions = study_conditions)


Starting processing...
Score reference...
Controls within SL score that are removed: 
0
---
Only GI cutoff is present...
Counts reference...
Number of double pairs: 37767
Number of controls: 614
Number of singles: 10550
Sequence reference...
Done! Returning...


In [61]:
print(db_inserts['score_ref'].columns)

Index(['gene_1', 'gene_2', 'study_origin', 'cell_line_origin', 'SL_score',
       'SL_score_cutoff', 'statistical_score', 'statistical_score_cutoff',
       'gene_pair', 'SL_or_not'],
      dtype='object')


In [62]:
print(db_inserts['sequence_ref'].columns)

Index(['sgRNA_guide_name', 'sgRNA_guide_seq', 'sgRNA_target_name',
       'study_origin'],
      dtype='object')


In [63]:
print(db_inserts['counts_ref'].columns)

Index(['guide_1', 'guide_2', 'gene_1', 'gene_2', 'count_replicates',
       'cell_line_origin', 'study_conditions', 'study_origin', 'target_type',
       'T0_counts', 'T0_replicate_names', 'TEnd_counts',
       'TEnd_replicate_names', 'gene_pair', 'gene_pair_orientation'],
      dtype='object')


In [9]:
# Finally, insert the data to the database
SLKB.insert_study_to_db(SLKB_engine, db_inserts)

Updating gene pairs with seperator |...
Final QC...
Beginning transaction...
Done sequence
Done counts
Done score
Successfully inserted!
Added Record stats...
Sequence insert: 247
Counts insert: 48931
Score insert: 1225
Done!


# Section 2 - Score Calculation

Here, we calculate the scores and add them to the database. First, we start by querying the data we just deposited.

In [10]:
# read the data

# experiment design
experiment_design = pd.read_sql_table('CDKO_EXPERIMENT_DESIGN', SLKB_engine, index_col = 'sgRNA_id')
experiment_design.reset_index(drop = True, inplace = True)
experiment_design.index.rename('sgRNA_id', inplace = True)

# counts
counts = pd.read_sql_table('CDKO_SGRNA_COUNTS', SLKB_engine, index_col = 'sgRNA_pair_id')
counts.reset_index(drop = True, inplace = True)
counts.index.rename('sgRNA_pair_id', inplace = True)

# scores
scores = pd.read_sql_table('CDKO_ORIGINAL_SL_RESULTS', SLKB_engine, index_col = 'id')
scores.reset_index(drop = True, inplace = True)
scores.index.rename('gene_pair_id', inplace = True)

# join the tables together
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_1_id', right_index = True, suffixes = ('', '_g1'))
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_2_id', right_index = True, suffixes = ('', '_g2'))
# rename
counts = counts.rename({'sgRNA_guide_name': 'sgRNA_guide_name_g1',
                        'sgRNA_guide_seq': 'sgRNA_guide_seq_g1',
                        'sgRNA_target_name': 'sgRNA_target_name_g1',
                        'study_origin_x': 'study_origin',
                        'cell_line_origin_x': 'cell_line_origin'}, axis = 1)

curr_study = '36060092'
curr_cl = '22RV1'
curr_counts = counts[(counts['study_origin'] == curr_study) & (counts['cell_line_origin'] == curr_cl)]

## Median B/NB Score

In [11]:
if not SLKB.check_if_added_to_table(curr_counts.copy(), 'MEDIAN_NB_SCORE', SLKB_engine):
    median_res = SLKB.run_median_scores(curr_counts.copy())
    SLKB.add_table_to_db(curr_counts.copy(), median_res['MEDIAN_NB_SCORE'], 'MEDIAN_NB_SCORE', SLKB_engine)
    if median_res['MEDIAN_B_SCORE'] is not None:
        SLKB.add_table_to_db(curr_counts.copy(), median_res['MEDIAN_B_SCORE'], 'MEDIAN_B_SCORE', SLKB_engine)

Checking if score already computed: MEDIAN_NB_SCORE
Running median scores...
Getting raw counts...
Filtering enabled... Condition: 35 counts
Filtered a total of 8134 out of 48931 sgRNAs.

---

Not full normalization...
Normalization enabled...
Current counts:
T0_1    34419.0
T0_2    34661.0
dtype: float64
Normalize based on a specific value... 44888.0 counts
Normalization enabled...
Current counts:
T12_1    55115.0
T12_2    65234.0
dtype: float64
Normalize based on a specific value... 44888.0 counts
Normalization enabled...
Current counts:
T0_1    4174896.0
T0_2    4200469.0
dtype: float64
Normalize based on a specific value... 4771163.0 counts
Normalization enabled...
Current counts:
T12_1    5341857.0
T12_2    6456429.0
dtype: float64
Normalize based on a specific value... 4771163.0 counts
Normalization enabled...
Current counts:
T0_1    877394.0
T0_2    878730.0
dtype: float64
Normalize based on a specific value... 1064517.0 counts
Normalization enabled...
Current counts:
T12_1    1

## sgRNA Derived B/NB Score

In [12]:
if not SLKB.check_if_added_to_table(curr_counts.copy(), 'SGRA_DERIVED_NB_SCORE', SLKB_engine):
    sgRNA_res = SLKB.run_sgrna_scores(curr_counts.copy())
    SLKB.add_table_to_db(curr_counts.copy(), sgRNA_res['SGRA_DERIVED_NB_SCORE'], 'SGRA_DERIVED_NB_SCORE', SLKB_engine)
    if sgRNA_res['SGRA_DERIVED_B_SCORE'] is not None:
        SLKB.add_table_to_db(curr_counts.copy(), sgRNA_res['SGRA_DERIVED_B_SCORE'], 'SGRA_DERIVED_B_SCORE', SLKB_engine)

Checking if score already computed: SGRA_DERIVED_NB_SCORE
Running sgrna derived score...
Getting raw counts...
Filtering enabled... Condition: 35 counts
Filtered a total of 8134 out of 48931 sgRNAs.

---

Not full normalization...
Normalization enabled...
Current counts:
T0_1    34419.0
T0_2    34661.0
dtype: float64
Normalize based on a specific value... 44888.0 counts
Normalization enabled...
Current counts:
T12_1    55115.0
T12_2    65234.0
dtype: float64
Normalize based on a specific value... 44888.0 counts
Normalization enabled...
Current counts:
T0_1    4174896.0
T0_2    4200469.0
dtype: float64
Normalize based on a specific value... 4771163.0 counts
Normalization enabled...
Current counts:
T12_1    5341857.0
T12_2    6456429.0
dtype: float64
Normalize based on a specific value... 4771163.0 counts
Normalization enabled...
Current counts:
T0_1    877394.0
T0_2    878730.0
dtype: float64
Normalize based on a specific value... 1064517.0 counts
Normalization enabled...
Current counts

## Horlbeck Score

In [14]:
if not SLKB.check_if_added_to_table(curr_counts.copy(), 'HORLBECK_SCORE', SLKB_engine):
    horlbeck_res = SLKB.run_horlbeck_score(curr_counts.copy(), curr_study = curr_study, curr_cl = curr_cl, store_loc = os.getcwd(), save_dir = 'HORLBECK_Files', do_preprocessing = True, re_run = False)
    SLKB.add_table_to_db(curr_counts.copy(), horlbeck_res['HORLBECK_SCORE'], 'HORLBECK_SCORE', SLKB_engine)

Checking if score already computed: HORLBECK_SCORE
Running horlbeck score...
Running preprocessing...
Getting raw counts...
Sorting gene pairs and guides based on ordering gene ordering...
For replicate 1
Total of 12 sgRNAs were filtered out of 222
For replicate 2
Total of 7 sgRNAs were filtered out of 222


/Users/gokbag.1/Documents/CodingProjectsLocal/SLKB-Analysis-Pipeline/SLKB_env/lib/python3.10/site-packages/pandas/core/groupby/ops.py:761: RuntimeWarning: Mean of empty slice
  res = f(group)


Calculating GI_Score_1...
Calculating GI_Score_2...


/Users/gokbag.1/Documents/CodingProjectsLocal/SLKB-Analysis-Pipeline/SLKB_env/lib/python3.10/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/gokbag.1/Documents/CodingProjectsLocal/SLKB-Analysis-Pipeline/SLKB_env/lib/python3.10/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


---------ADDING-TO-DB---------
Processing table for: HORLBECK_SCORE
Beginning transaction...
Successfully inserted!
Added Record stats...
Score insert: 1225


## GEMINI Score

In [15]:
cmd_params = []#['module load R/4.1.0']
if not SLKB.check_if_added_to_table(curr_counts.copy(), 'GEMINI_SCORE', SLKB_engine):
    gemini_res = SLKB.run_gemini_score(curr_counts.copy(), curr_study = curr_study, curr_cl = curr_cl, store_loc = os.getcwd(), save_dir = 'GEMINI_Files', command_line_params = cmd_params, re_run = False)
    SLKB.add_table_to_db(curr_counts.copy(), gemini_res['GEMINI_SCORE'], 'GEMINI_SCORE', SLKB_engine)

Checking if score already computed: GEMINI_SCORE
Running gemini score...
Getting raw counts...
Running GEMINI...
Finished running GEMINI!
---------ADDING-TO-DB---------
Processing table for: GEMINI_SCORE
Beginning transaction...
Successfully inserted!
Added Record stats...
Score insert: 1225


## MAGeCK Score

In [ ]:
cmd_params = ['conda activate myEnv']
if not SLKB.check_if_added_to_table(curr_counts.copy(), 'MAGECK_SCORE', SLKB_engine):
    mageck_res = SLKB.run_mageck_score(curr_counts.copy(), curr_study = curr_study, curr_cl = curr_cl, store_loc = os.getcwd(), save_dir = 'MAGECK_Files', command_line_params = cmd_params,re_run = False)
    SLKB.add_table_to_db(curr_counts.copy(), mageck_res['MAGECK_SCORE'], 'MAGECK_SCORE', SLKB_engine)

# Section 3 - Query Results

Finally, we can query the data to produce the calculation table.

In [16]:
# read the data

# experiment design
experiment_design = pd.read_sql_table('CDKO_EXPERIMENT_DESIGN', SLKB_engine, index_col = 'sgRNA_id')
experiment_design.drop(['study_origin'], axis = 1, inplace = True)
experiment_design.reset_index(drop = True, inplace = True)
experiment_design.index.rename('sgRNA_id', inplace = True)

# counts
counts = pd.read_sql_table('CDKO_SGRNA_COUNTS', SLKB_engine, index_col = 'sgRNA_pair_id')
counts.reset_index(drop = True, inplace = True)
counts.index.rename('sgRNA_pair_id', inplace = True)

# scores
scores = pd.read_sql_table('CDKO_ORIGINAL_SL_RESULTS', SLKB_engine, index_col = 'gene_pair_id')

# join the tables together
counts = counts.merge(scores, how = 'left', left_on = 'gene_pair_id', right_index = True)
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_1_id', right_index = True, suffixes = ('', '_g1'))
counts = counts.merge(experiment_design, how = 'left', left_on = 'guide_2_id', right_index = True, suffixes = ('', '_g2'))
# rename
counts = counts.rename({'sgRNA_guide_name': 'sgRNA_guide_name_g1',
                        'sgRNA_guide_seq': 'sgRNA_guide_seq_g1',
                        'sgRNA_target_name': 'sgRNA_target_name_g1',
                        'study_origin_x': 'study_origin',
                        'cell_line_origin_x': 'cell_line_origin'}, axis = 1)

experiment_design = pd.read_sql_table('CDKO_EXPERIMENT_DESIGN', SLKB_engine, index_col = 'sgRNA_id')
experiment_design.reset_index(drop = True, inplace = True)
experiment_design.index.rename('sgRNA_id', inplace = True)

# tables to obtain the data from
all_results_tables = ['HORLBECK_SCORE', 
                      'MAGECK_SCORE', 
                      'SGRA_DERIVED_NB_SCORE', 
                      'SGRA_DERIVED_B_SCORE',
                      'MEDIAN_NB_SCORE', 
                      'MEDIAN_B_SCORE',
                      'GEMINI_SCORE']

available_studies = sorted(set(counts['study_origin']))

#################

all_scores = []
for curr_study in available_studies:
    print('Working on study: ' + curr_study)

    # get study counts and seq
    study_counts = counts.loc[counts['study_origin'] == curr_study].copy()

    curr_seq_ids = np.array(sorted(list(set(study_counts['guide_1_id'].tolist() + study_counts['guide_2_id'].tolist()))))
    study_sequences = experiment_design.loc[curr_seq_ids]

    # the analysis runs for each individual cell line
    available_cell_lines = set(study_counts['cell_line_origin'])


    for curr_cl in available_cell_lines:
        # store results here
        study_scores = []
    
        print('Working on cell line: ' + curr_cl)
        curr_counts = study_counts.loc[study_counts['cell_line_origin'] == curr_cl].copy()
        
        for table_name in all_results_tables:
            # add the result of the table to the list
            study_scores.append(SLKB.query_result_table(curr_counts.copy(), table_name, curr_study, curr_cl, SLKB_engine))
    
        # remove duplicate annotation columns
        study_scores = pd.concat(study_scores, axis = 1, ignore_index = False)
        study_scores = study_scores.loc[:,~study_scores.columns.duplicated(keep = 'last')].copy()
        
        # make sure the annotations are all filled
        #study_scores['gene_pair'] = study_scores.index
        study_scores['study_origin'] = curr_study
        study_scores['cell_line_origin'] = curr_cl
        
        # reset the index, gene_pair -> id
        study_scores.reset_index(drop = True, inplace = True)

        # add to big table 
        all_scores.append(study_scores)
    
    print('-----')
    
print('Done getting all data!')
    
# combine the scores at the end
all_scores = pd.concat(all_scores, axis = 0, ignore_index = True)

# add individual genes
all_scores['gene_1'] = [i.split('|')[0] for i in all_scores['gene_pair']]
all_scores['gene_2'] = [i.split('|')[1] for i in all_scores['gene_pair']]

# sort such that all annotations are at the front
all_columns = sorted(list(all_scores.columns))
annotation_columns = ['gene_pair', 'gene_1', 'gene_2', 'study_origin', 'cell_line_origin']

# get the final scores
all_scores = all_scores.loc[:, annotation_columns + [i for i in all_columns if i not in annotation_columns]]

Working on study: 36060092
Working on cell line: 22RV1
Accessing table: HORLBECK_SCORE
Available gene pairs: 1225
Accessing table: MAGECK_SCORE
Available gene pairs: 0
Accessing table: SGRA_DERIVED_NB_SCORE
Available gene pairs: 1225
Accessing table: SGRA_DERIVED_B_SCORE
Available gene pairs: 1225
Accessing table: MEDIAN_NB_SCORE
Available gene pairs: 1225
Accessing table: MEDIAN_B_SCORE
Available gene pairs: 1225
Accessing table: GEMINI_SCORE
Available gene pairs: 1225
-----
Done getting all data!


In [17]:
all_scores.head(15)

,gene_pair,gene_1,gene_2,study_origin,cell_line_origin,GEMINI_SCORE_SL_score_SensitiveLethality,GEMINI_SCORE_SL_score_SensitiveRecovery,GEMINI_SCORE_SL_score_Strong,HORLBECK_SCORE_SL_score,HORLBECK_SCORE_standard_error,...,MAGECK_SCORE_Z_SL_score,MAGECK_SCORE_standard_error,MEDIAN_B_SCORE_SL_score,MEDIAN_B_SCORE_Z_SL_score,MEDIAN_B_SCORE_standard_error,MEDIAN_NB_SCORE_SL_score,MEDIAN_NB_SCORE_Z_SL_score,MEDIAN_NB_SCORE_standard_error,SGRA_DERIVED_B_SCORE_SL_score,SGRA_DERIVED_NB_SCORE_SL_score
0,AKT3|AR,AKT3,AR,36060092,22RV1,-0.091424,NaN,-0.091424,-0.589468,0.258965,...,NaN,NaN,-0.062565,-1.099934,0.056881,-0.022646,-0.398139,0.056881,-2.490022,-1.760639
1,AKT3|AURKA,AKT3,AURKA,36060092,22RV1,-0.046752,NaN,-0.046752,-0.014408,0.156112,...,NaN,NaN,0.020551,0.200761,0.102366,0.060470,0.590720,0.102366,0.346373,0.496084
2,AKT3|BMP6,AKT3,BMP6,36060092,22RV1,-0.035922,NaN,-0.035922,-0.304914,0.168399,...,NaN,NaN,-0.094960,-2.147660,0.044215,-0.055041,-1.244839,0.044215,-1.648458,-0.954725
3,AKT3|CCNE2,AKT3,CCNE2,36060092,22RV1,-0.045427,NaN,-0.045427,-0.262514,0.123153,...,NaN,NaN,-0.052437,-1.201618,0.043639,-0.012519,-0.286871,0.043639,0.015597,0.616933
4,AKT3|CDC6,AKT3,CDC6,36060092,22RV1,0.002442,NaN,0.002442,0.015164,0.186259,...,NaN,NaN,0.005039,0.046173,0.109140,0.044958,0.411928,0.109140,0.079230,0.253455
5,AKT3|CDK2,AKT3,CDK2,36060092,22RV1,-0.082281,NaN,-0.082281,-0.015867,0.121892,...,NaN,NaN,0.006520,0.099859,0.065290,0.046438,0.711268,0.065290,1.357099,2.168888
6,AKT3|CTNNB1,AKT3,CTNNB1,36060092,22RV1,-0.126063,NaN,-0.126063,-0.368898,0.151781,...,NaN,NaN,-0.043282,-1.084114,0.039923,-0.003363,-0.084237,0.039923,-2.260286,-1.494126
7,AKT3|DHFR,AKT3,DHFR,36060092,22RV1,-0.072729,NaN,-0.072729,-0.207039,0.126963,...,NaN,NaN,0.068544,0.687798,0.099658,0.108463,1.088355,0.099658,-1.010516,0.270471
8,AKT3|ETF1,AKT3,ETF1,36060092,22RV1,NaN,0.046196,-0.132287,0.167150,0.218769,...,NaN,NaN,-0.203226,-0.950267,0.213862,-0.163307,-0.763611,0.213862,-0.366060,-0.069083
9,AKT3|EZH2,AKT3,EZH2,36060092,22RV1,-0.052789,NaN,-0.052789,0.016892,0.199305,...,NaN,NaN,0.021071,0.351438,0.059958,0.060990,1.017218,0.059958,-0.024138,0.495638


In [18]:
# save to current directory
all_scores.to_csv(os.path.join(os.getcwd(), 'all_scores.csv'))